In [35]:
import numpy as np
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [20]:
dataset = pd.read_csv("penbased-5an-nn.csv",header= None)
dataset.columns = ['AT1','AT2','AT3','AT4','AT5','AT6','AT7','AT8',
                   'AT9','AT10','AT11','AT12','AT13','AT14','AT15','AT16','CLASS']

In [21]:
dataset

,AT1,AT2,AT3,AT4,AT5,AT6,AT7,AT8,AT9,AT10,AT11,AT12,AT13,AT14,AT15,AT16,CLASS
0,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
1,99,80,63,100,25,76,79,68,100,62,97,23,54,0,0,16,9
2,0,73,19,99,72,100,70,73,32,48,5,18,46,0,100,72,2
3,12,77,20,62,78,40,50,0,1,17,0,64,23,98,100,100,5
4,0,98,36,42,80,85,68,42,56,0,25,23,50,37,100,32,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10987,86,100,38,80,12,51,0,18,94,0,100,13,62,30,7,18,6
10988,0,92,48,100,76,74,70,41,57,9,38,0,46,80,100,20,7
10989,0,82,26,98,59,100,67,78,55,53,49,27,65,6,15,0,2
10990,5,90,55,100,78,84,90,65,71,51,100,23,52,5,0,0,3


In [22]:
dataset.shape

(10992, 17)

In [ ]:
x= dataset[['AT1','AT2','AT3','AT4','AT5','AT6','AT7','AT8',
                   'AT9','AT10','AT11','AT12','AT13','AT14','AT15','AT16']]
x

In [ ]:
y = dataset['CLASS']
y

In [ ]:
# test_size: what proportion of original data is used for test set
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [24]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
#Feature Selection
model = LogisticRegression()
rfe = RFE(model, 3)
fit = rfe.fit(x, y)

print("Number of Features: ", fit.n_features_)
print("Selected Features: ", fit.support_)
print("Feature Ranking: ", fit.ranking_) 

Number of Features:  3
Selected Features:  [ True False False False False False False False False  True False False
 False False False  True]
Feature Ranking:  [ 1  7 12 11  6 14  4  8  5  1  9 10 13  3  2  1]


In [30]:
# Tinh do chinh xac model theo CAC ML Models sau:
models = [
    LogisticRegression(max_iter=200),
    GaussianNB(),
    SVC(),
    RandomForestClassifier(n_estimators=200),
    DecisionTreeClassifier(),
    KNeighborsClassifier(n_neighbors=3)
]

#so lan lap kiem chung 
CV = 10
entries = []
i = 0

for model in models:
    scores_train = []
    scores_test = []
    times = []
    abs_score = []
    for j in range(CV):
        X_train,X_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)
        t1 = datetime.datetime.now()
        model_name = model.__class__.__name__
        model.fit(X_train,y_train)
        t2 = datetime.datetime.now()
        d = round((t2 - t1).microseconds/1000,1)
        score_train = model.score(X_train,y_train)
        score_test = model.score(X_test,y_test)
        abs_score = abs(score_train - score_test)
        
        scores_train.append(score_train)
        scores_test.append(score_test)
        times.append(d)
        
    print(model.__class__.__name__, scores_test)
    entries.append([model_name, np.array(scores_train).mean(),
                   np.array(scores_test).mean(), np.array(abs_score).mean(),
                   np.array(times).mean()])
    
# in ket qua i+=1
cv_df = pd.DataFrame(entries, columns=['model_name','score_train_mean','score_test_mean','abs|score|','time_mean'])
        

LogisticRegression [0.8485675306957708, 0.842655752614825, 0.846293769895407, 0.8467485220554798, 0.8485675306957708, 0.8422010004547522, 0.8476580263756253, 0.8476580263756253, 0.8458390177353342, 0.8472032742155525]
GaussianNB [0.7885402455661664, 0.804456571168713, 0.7871759890859482, 0.7976352887676217, 0.8053660754888585, 0.8030923146884947, 0.7939972714870396, 0.7844474761255116, 0.8003638017280582, 0.8067303319690768]
SVC [0.9590723055934516, 0.958162801273306, 0.9577080491132333, 0.9595270577535243, 0.9590723055934516, 0.9640745793542519, 0.9631650750341064, 0.959981809913597, 0.9604365620736699, 0.9627103228740337]
RandomForestClassifier [0.9749886311959982, 0.979081400636653, 0.975443383356071, 0.9777171441564347, 0.9686221009549796, 0.9758981355161437, 0.9731696225557072, 0.9786266484765803, 0.9740791268758526, 0.9699863574351978]
DecisionTreeClassifier [0.9081400636653024, 0.8844929513415188, 0.9113233287858117, 0.8954070031832652, 0.8967712596634834, 0.8890404729422465, 0.

In [31]:
cv_df

,model_name,score_train_mean,score_test_mean,abs|score|,time_mean
0,LogisticRegression,0.852417,0.846339,0.004611,245.33
1,GaussianNB,0.804322,0.797181,0.003023,6.10
2,SVC,0.971830,0.960391,0.008631,754.06
3,RandomForestClassifier,1.000000,0.974761,0.030014,536.25
4,DecisionTreeClassifier,1.000000,0.900318,0.093224,72.70
5,KNeighborsClassifier,0.979552,0.962983,0.014204,2.40


## SVM

In [37]:
clf_svm = svm.SVC()
clf_svm.fit(X_train, y_train)

SVC()

In [38]:
y_pred_svm = clf_svm.predict(X_test)
y_pred_svm

array([9, 9, 9, ..., 7, 3, 0], dtype=int64)

In [39]:
print("Accuracy is ", accuracy_score(y_test, y_pred_svm)*100, "%")

Accuracy is  96.1800818553888 %


In [40]:
pd.DataFrame(confusion_matrix(y_test, y_pred_svm))

,0,1,2,3,4,5,6,7,8,9
0,220,0,0,0,0,0,1,0,1,0
1,0,197,10,3,2,0,0,2,0,0
2,0,9,207,1,0,0,0,2,0,0
3,0,7,0,202,0,1,0,0,1,1
4,0,0,0,1,219,0,2,0,1,1
5,0,3,0,2,1,195,0,0,4,2
6,1,1,0,0,1,2,227,0,0,0
7,0,7,0,3,1,0,0,224,2,1
8,1,1,0,1,0,0,0,1,215,0
9,0,0,0,0,0,2,0,1,0,209


In [41]:
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       222
           1       0.88      0.92      0.90       214
           2       0.95      0.95      0.95       219
           3       0.95      0.95      0.95       212
           4       0.98      0.98      0.98       224
           5       0.97      0.94      0.96       207
           6       0.99      0.98      0.98       232
           7       0.97      0.94      0.96       238
           8       0.96      0.98      0.97       219
           9       0.98      0.99      0.98       212

    accuracy                           0.96      2199
   macro avg       0.96      0.96      0.96      2199
weighted avg       0.96      0.96      0.96      2199



## RandomForest

In [43]:
clf_rf = RandomForestClassifier(n_estimators=200)
clf_rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200)

In [44]:
y_pred_rf = clf_rf.predict(X_test)
y_pred_rf

array([9, 9, 9, ..., 7, 3, 0], dtype=int64)

In [45]:
print("Accuracy is ", accuracy_score(y_test, y_pred_rf)*100, "%")

Accuracy is  97.5443383356071 %


In [46]:
pd.DataFrame(confusion_matrix(y_test, y_pred_rf))

,0,1,2,3,4,5,6,7,8,9
0,220,0,0,0,0,0,0,0,2,0
1,0,200,12,1,0,0,0,1,0,0
2,0,0,217,2,0,0,0,0,0,0
3,0,3,0,206,0,0,0,1,1,1
4,1,0,0,0,220,0,1,0,0,2
5,0,1,0,1,0,199,0,0,3,3
6,0,0,0,0,1,1,230,0,0,0
7,0,4,1,1,1,0,0,229,2,0
8,4,0,0,0,0,0,0,1,214,0
9,0,0,0,0,1,0,0,1,0,210


In [47]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       222
           1       0.96      0.93      0.95       214
           2       0.94      0.99      0.97       219
           3       0.98      0.97      0.97       212
           4       0.99      0.98      0.98       224
           5       0.99      0.96      0.98       207
           6       1.00      0.99      0.99       232
           7       0.98      0.96      0.97       238
           8       0.96      0.98      0.97       219
           9       0.97      0.99      0.98       212

    accuracy                           0.98      2199
   macro avg       0.98      0.98      0.98      2199
weighted avg       0.98      0.98      0.98      2199



# PCA

In [48]:
pca = PCA(.95)
pca.fit(X_train)

PCA(n_components=0.95)

In [49]:
pca.n_components_

11

In [50]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

## SVM (PCA)

In [51]:
clf_svm_pca = svm.SVC()
clf_svm_pca.fit(X_train_pca, y_train)

SVC()

In [52]:
y_pred_svm_pca = clf_svm_pca.predict(X_test_pca)
y_pred_svm_pca

array([9, 9, 9, ..., 7, 3, 0], dtype=int64)

In [53]:
print("Accuracy is ", accuracy_score(y_test, y_pred_svm_pca)*100, "%")

Accuracy is  95.58890404729422 %


In [54]:
pd.DataFrame(confusion_matrix(y_test, y_pred_svm_pca))

,0,1,2,3,4,5,6,7,8,9
0,220,0,0,0,0,0,1,0,1,0
1,0,194,13,4,2,0,0,0,0,1
2,0,7,206,2,0,0,0,3,1,0
3,0,8,0,200,0,2,0,1,0,1
4,0,1,0,0,218,0,3,0,1,1
5,0,3,0,2,0,194,0,0,4,4
6,1,1,0,0,3,1,226,0,0,0
7,0,7,2,2,1,0,0,223,3,0
8,1,0,1,1,0,0,1,1,214,0
9,0,1,0,1,1,1,0,1,0,207


In [55]:
print(classification_report(y_test, y_pred_svm_pca))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       222
           1       0.87      0.91      0.89       214
           2       0.93      0.94      0.93       219
           3       0.94      0.94      0.94       212
           4       0.97      0.97      0.97       224
           5       0.98      0.94      0.96       207
           6       0.98      0.97      0.98       232
           7       0.97      0.94      0.96       238
           8       0.96      0.98      0.97       219
           9       0.97      0.98      0.97       212

    accuracy                           0.96      2199
   macro avg       0.96      0.96      0.96      2199
weighted avg       0.96      0.96      0.96      2199



## RandomForest (PCA)

In [56]:
clf_rf_pca = RandomForestClassifier(n_estimators=200)
clf_rf_pca.fit(X_train_pca, y_train)

RandomForestClassifier(n_estimators=200)

In [57]:
y_pred_rf_pca = clf_rf_pca.predict(X_test_pca)
y_pred_rf_pca

array([9, 9, 9, ..., 7, 3, 0], dtype=int64)

In [58]:
print("Accuracy is ", accuracy_score(y_test, y_pred_rf_pca)*100, "%")

Accuracy is  94.63392451114143 %


In [59]:
pd.DataFrame(confusion_matrix(y_test, y_pred_rf_pca))

,0,1,2,3,4,5,6,7,8,9
0,220,0,0,0,0,0,0,0,2,0
1,0,188,16,5,2,0,0,1,0,2
2,0,6,209,2,0,1,0,1,0,0
3,0,5,0,200,1,1,0,1,0,4
4,1,2,0,0,214,0,4,1,1,1
5,0,1,2,1,0,193,1,0,4,5
6,2,0,0,0,4,1,224,0,1,0
7,1,9,5,1,1,1,0,217,3,0
8,1,1,1,1,0,0,1,4,210,0
9,0,2,0,2,0,2,0,0,0,206


In [60]:
print(classification_report(y_test, y_pred_rf_pca))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       222
           1       0.88      0.88      0.88       214
           2       0.90      0.95      0.92       219
           3       0.94      0.94      0.94       212
           4       0.96      0.96      0.96       224
           5       0.97      0.93      0.95       207
           6       0.97      0.97      0.97       232
           7       0.96      0.91      0.94       238
           8       0.95      0.96      0.95       219
           9       0.94      0.97      0.96       212

    accuracy                           0.95      2199
   macro avg       0.95      0.95      0.95      2199
weighted avg       0.95      0.95      0.95      2199



### Dựa vào Accuracy score chọn Random forest và không cần PCA 